<a href="https://colab.research.google.com/github/ArefPhD/General-training/blob/main/RecreatinganypictureGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import division, print_function, absolute_import

import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf

from tensorflow import keras

In [ ]:
# Import MNIST data
tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

In [ ]:
import tensorflow_datasets
mnist = tensorflow_datasets.load('mnist')

In [ ]:
L=mnist.get('train')

In [ ]:
# Training Params
num_steps = 500
batch_size = 128
learning_rate = 0.0002

# Network Params
image_dim = 784 # 28*28 pixels
gen_hidden_dim = 256
disc_hidden_dim = 256
noise_dim = 100 # Noise data points

# A custom initialization (see Xavier Glorot init)
def glorot_init(shape):
    return tf.random.normal(shape=shape, stddev=1. / tf.sqrt(shape[0] / 2.))

In [ ]:
# Store layers weight & bias
weights = {
    'gen_hidden1': tf.Variable(glorot_init([100, 256])),
    'gen_out': tf.Variable(glorot_init([256, 784])),
    'disc_hidden1': tf.Variable(glorot_init([784, 256])),
    'disc_out': tf.Variable(glorot_init([256, 1])),
}
biases = {
    'gen_hidden1': tf.Variable(tf.zeros([256])),
    'gen_out': tf.Variable(tf.zeros([784])),
    'disc_hidden1': tf.Variable(tf.zeros([256])),
    'disc_out': tf.Variable(tf.zeros([1])),
}

In [ ]:
# Generator
def generator(x):
    hidden_layer = tf.matmul(x, weights['gen_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['gen_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['gen_out'])
    out_layer = tf.add(out_layer, biases['gen_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer


# Discriminator
def discriminator(x):
    hidden_layer = tf.matmul(x, weights['disc_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['disc_out'])
    out_layer = tf.add(out_layer, biases['disc_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

# Build Networks
# Network Inputs


In [ ]:
tf.disable_v2_behavior()


In [ ]:
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name='input_noise')
disc_input = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input')


# Build Generator Network
gen_sample = generator(gen_input)

# Build 2 Discriminator Networks (one from noise input, one from generated samples)
A = discriminator(disc_input)
B = discriminator(gen_sample)

# Build Loss
gen_loss = -tf.reduce_mean(tf.log(B))
disc_loss = -tf.reduce_mean(tf.log(A) + tf.log(1. - B))

# Build Optimizers
optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)

# Training Variables for each optimizer
# By default in TensorFlow, all variables are updated by each optimizer, so we
# need to precise for each one of them the specific variables to update.
# Generator Network Variables
gen_vars = [weights['gen_hidden1'], weights['gen_out'],
            biases['gen_hidden1'], biases['gen_out']]
# Discriminator Network Variables
disc_vars = [weights['disc_hidden1'], weights['disc_out'],
            biases['disc_hidden1'], biases['disc_out']]

# Create training operations
train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
import tensorflow_datasets as tfds
import numpy as np

dataset = tfds.load('mnist', split=['test'], as_supervised=True)

array = np.vstack(tfds.as_numpy(dataset[0]))

X_train = np.array(list(map(lambda x: x[0], array)))
y_train = np.array(list(map(lambda x: x[1], array)))


In [ ]:
from google.colab import files
upload=files.upload()

In [ ]:
import numpy as np
import scipy.io

L=scipy.io.loadmat('1jpg (1).mat')


In [ ]:
S=L.get('img')

In [ ]:
L=S
L=tf.reshape(L,[28, 28, 1])
L=K.eval(L)

In [ ]:
pil_img = tf.keras.preprocessing.image.array_to_img(L)
pil_img

In [ ]:
from google.colab import files
upload=files.upload()

In [ ]:
from PIL import Image

In [ ]:
Im=Image.open('image4001.png')

In [ ]:
L.shape

In [ ]:

X_train[0]=L


In [ ]:
pil_img.shape

In [ ]:
from keras import backend as K

In [ ]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)

# Training
for i in range(1, num_steps+1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    batch_x = X_train[0]
    batch_x=tf.reshape(batch_x, [1, 784])
    batch_x= K.eval(batch_x)
    # Generate noise to feed to the generator
    z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])

    # Train
    feed_dict = {disc_input: batch_x, gen_input: z}
    _, _, gl, dl = sess.run([train_gen, train_disc, gen_loss, disc_loss],
                            feed_dict=feed_dict)
    if i % 2 == 0 or i == 1:
        print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))

In [ ]:

# Testing
# Generate images from noise, using the generator network.
n = 6
canvas = np.empty((28 * n, 28 * n))
for i in range(n):
    # Noise input.
    z = np.random.uniform(-1., 1., size=[n, noise_dim])
    # Generate image from noise.
    g = sess.run(gen_sample, feed_dict={gen_input: z})
    # Reverse colours for better display
    g = -1 * (g - 1)
    for j in range(n):
        # Draw the generated digits
        canvas[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = g[j].reshape([28, 28])

plt.figure(figsize=(n, n))
plt.imshow(canvas, origin="upper", cmap="gray")
plt.show()